# Asynchronus Advantage Actor-Critic (A3C) on a Snake Robot

This iPython notebook includes an implementation of the [A3C algorithm](https://arxiv.org/pdf/1602.01783.pdf), based on Arthur Juliani's A3C code. For more information on A3C, see the accompanying [Medium post](https://medium.com/p/c88f72a5e9f2/edit).

This version runs A3C with 6 workers on the offline database "p_experiments.snake" (which needs to be downloaded separately, see README.md), which was obtained on a real snake robot running the state-of-the-art compliant controller with 6 windows.

While training is taking place, statistics on agent performance are available from Tensorboard. To launch it use:

`tensorboard --logdir=worker_0:'./train_W_0',worker_1:'./train_W_1',worker_2:'./train_W_2',worker_3:'./train_W_3',worker_4:'./train_W_4',worker_5:'./train_W_5'`

In [1]:
import pickle
with open("p_experiments.snake","rb") as f:
    exps = pickle.load(f)

In [2]:
from __future__ import division

import SnakeEnvironment
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import sys, os
print(sys.version_info)
import multiprocessing
import threading
import shutil

sys.version_info(major=3, minor=4, micro=6, releaselevel='final', serial=0)



### Parameters

In [3]:
OUTPUT_GRAPH = True
LOG_DIR = './log'
N_WORKERS = 6 #multiprocessing.cpu_count()
MAX_GLOBAL_EP = 50000
GLOBAL_NET_SCOPE = 'Global_Net'
UPDATE_GLOBAL_ITER = 89
GAMMA = 0.995
ENTROPY_BETA = 0.01
LR_A = 1e-4    # learning rate for actor
LR_C = 1e-4    # learning rate for critic
GLOBAL_REWARD = []
GLOBAL_EP = 0
model_path = './model_offlineA3C'
load_model = False
episode_length = 89


s_size = 7
a_size = 9

os.system('rm -Rf log train_W* '+model_path)
os.makedirs(model_path)

## A3C Approach

### Implementing the Actor-Critic network

In [4]:
class ACNet(object):
    def __init__(self, scope, globalAC=None):

        if scope == GLOBAL_NET_SCOPE:   # Only need parameters of global network
            with tf.variable_scope(scope):
                self.s = tf.placeholder(tf.float32, [None, s_size], 'S')
                self._build_net()
                self.a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
                self.c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
        else:   # Uselocal net to calculate losses
            with tf.variable_scope(scope):
                self.s = tf.placeholder(tf.float32, [None, s_size], 'S')
                self.a_his = tf.placeholder(tf.int32, [None, ], 'A')
                self.v_target = tf.placeholder(tf.float32, [None, 1], 'Vtarget')
                
                #Get policy(a_prob) and value(v) from actor, critic net
                self.a_prob, self.v = self._build_net()

                td = tf.subtract(self.v_target, self.v, name='TD_error')
                with tf.name_scope('q_loss'):
                    self.c_loss = tf.reduce_mean(tf.square(td))

                with tf.name_scope('a_loss'):
                    #use clip to avoid log(0): tf.clip_by_value(self.policy, 1e-20, 1.0)
                    log_prob = tf.reduce_sum(tf.log(tf.clip_by_value(self.a_prob, 1e-20, 1.0)) * tf.one_hot(self.a_his, a_size, dtype=tf.float32), axis=1, keep_dims=True)
                    exp_v = log_prob * td
                    #Found someone using entropy to encourage exploration
                    #larger entropy means more stochastic actions
                    entropy = -tf.reduce_sum(self.a_prob * tf.log(tf.clip_by_value(self.a_prob, 1e-20, 1.0)), axis=1, keep_dims=True)
                    self.exp_v = ENTROPY_BETA * entropy + exp_v
                    self.a_loss = tf.reduce_mean(-self.exp_v)

                with tf.name_scope('local_grad'):
                    self.a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/actor')
                    self.c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope + '/critic')
                    self.a_grads = tf.gradients(self.a_loss, self.a_params)
                    self.c_grads = tf.gradients(self.c_loss, self.c_params)
                
            # Syncronization
            with tf.name_scope('sync'):
                #assign global parameters to local parameters
                with tf.name_scope('pull'):
                    self.pull_a_params_op = [local_param.assign(global_param) for local_param, global_param in zip(self.a_params, globalAC.a_params)]
                    self.pull_c_params_op = [local_param.assign(global_param) for local_param, global_param in zip(self.c_params, globalAC.c_params)]
                #update params of global net by pushing the calculated gradients of local net to global net    
                with tf.name_scope('push'):
                    self.update_a_op = trainer_A.apply_gradients(zip(self.a_grads, globalAC.a_params))
                    self.update_c_op = trainer_C.apply_gradients(zip(self.c_grads, globalAC.c_params))

    def _build_net(self):
        w_init = tf.random_normal_initializer(0., .1)
        with tf.variable_scope('actor'):
            layer4 = tf.layers.dense(self.s, a_size, tf.nn.relu6, kernel_initializer=w_init, name='layer4')
            layer3 = tf.layers.dense(layer4, a_size, tf.nn.relu6, kernel_initializer=w_init, name='layer3')
            layer2 = tf.layers.dense(layer3, a_size, tf.nn.relu6, kernel_initializer=w_init, name='layer2')
            a_prob = tf.layers.dense(layer2, a_size, tf.nn.softmax, kernel_initializer=w_init, name='ap')
        with tf.variable_scope('critic'):
            v4 = tf.layers.dense(self.s, 1, tf.nn.relu6, kernel_initializer=w_init, name='v4')
            v3 = tf.layers.dense(v4, 1, tf.nn.relu6, kernel_initializer=w_init, name='v3')
            v2 = tf.layers.dense(v3, 1, tf.nn.relu6, kernel_initializer=w_init, name='v2')
            v = tf.layers.dense(v2, 1, tf.nn.relu6, kernel_initializer=w_init, name='v')
        return a_prob, v

    def update_global(self, feed_dict):  # run by a local
        a_grads,c_grads,c_loss,a_loss,_,_ = SESS.run([self.a_grads,self.c_grads,self.c_loss, self.a_loss,self.update_a_op, self.update_c_op], feed_dict)# local grads applies to global net
        return a_loss,c_loss,a_grads,c_grads
    def pull_global(self):  # run by a local
        SESS.run([self.pull_a_params_op, self.pull_c_params_op])

    def choose_action(self, s):  # run by a local
        prob_weights = SESS.run(self.a_prob, feed_dict={self.s: np.array(s)})
        action = np.random.choice(range(prob_weights.shape[1]),
                                  p=prob_weights.ravel())  # select action w.r.t the actions prob
        return action

## Worker Agent

In [5]:
class Worker(object):
    def __init__(self, name, workerID, globalAC):
        self.workerID = workerID
        self.env = SnakeEnvironment.SnakeEnv(exps,workerID,89)
        self.name = name
        self.AC = ACNet(name, globalAC)
        self.model_path = model_path
        self.summary_writer = tf.summary.FileWriter("train_"+str(self.name))        

    def work(self):
        global GLOBAL_REWARD, GLOBAL_EP
        total_step = 1
        buffer_s, buffer_a, buffer_r = [], [], []
        while not COORD.should_stop() and GLOBAL_EP < MAX_GLOBAL_EP:
            s,_,_ = self.env.reset()
            ep_r = 0
            ep_step=0
            ep_action_counter=0
            a_loss, c_loss = [], []            
            while True:
                a,_ = self.env.action_space.sample()
                s1, r, done,_ = self.env.step(a)
                ep_r += r
                buffer_s.append(s)
                buffer_a.append(a)
                buffer_r.append(r)

                if total_step % UPDATE_GLOBAL_ITER == 0:   # update global and assign to local net
                    if done:
                        v_s1 = 0   # terminal
                    else:
                        v_s1 = SESS.run(self.AC.v, {self.AC.s: np.array(s1)})[0, 0]
                        assert not(np.any(np.isnan(v_s1)))
                    buffer_v_target = []
                    for r in buffer_r[::-1]:    # reverse buffer r
                        v_s1 = r + GAMMA * v_s1
                        buffer_v_target.append(v_s1)
                    buffer_v_target.reverse()

                    buffer_s, buffer_a, buffer_v_target = np.vstack(buffer_s), np.array(buffer_a), np.vstack(buffer_v_target)
                    feed_dict = {
                        self.AC.s: buffer_s,
                        self.AC.a_his: buffer_a,
                        self.AC.v_target: buffer_v_target,
                    }
                    a_l,c_l,a_grads,c_grads = self.AC.update_global(feed_dict)
                    a_loss.append(a_l)
                    c_loss.append(c_l)

                    buffer_s, buffer_a, buffer_r = [], [], []
                    self.AC.pull_global()

                s = s1
                total_step += 1
                ep_step +=1
                if ep_step == episode_length:
                    GLOBAL_REWARD.append(ep_r)

                    print(
                        self.name,
                        "Ep:", GLOBAL_EP,
                        "| Ep_r: %i" % GLOBAL_REWARD[-1]
                          )
                    if GLOBAL_EP % 100 == 0:
                        saver.save(SESS,self.model_path+'/model-'+str(GLOBAL_EP)+'.cptk')
                        print ("Saved Model")
                    if GLOBAL_EP % 5 ==0:
                        mean_reward = np.mean(GLOBAL_REWARD[-5:])
                        mean_c_loss = np.max(np.mean(c_loss[-5:]))
                        mean_a_loss = np.max(np.mean(a_loss[-5:]))
                        summary=tf.Summary()
                        summary.value.add(tag='Perf/Reward', simple_value=float(mean_reward))
                        summary.value.add(tag='Losses/Value Loss', simple_value=float(mean_c_loss))
                        summary.value.add(tag='Losses/Policy Loss', simple_value=float(mean_a_loss))
                        self.summary_writer.add_summary(summary, GLOBAL_EP)

                        self.summary_writer.flush()
                    GLOBAL_EP += 1
                    
                    break

### Training the network

In [ ]:
if __name__ == "__main__":
    os.system("rm -r train_W*")
    SESS = tf.Session()

    with tf.device("/cpu:0"):
        trainer_A = tf.train.AdamOptimizer(LR_A, name='RMSPropA')
        trainer_C = tf.train.AdamOptimizer(LR_C, name='RMSPropC')
        GLOBAL_AC = ACNet(GLOBAL_NET_SCOPE)  # we only need its params
        workers = []
        # Create worker
        for i in range(N_WORKERS):#N_WORKERS
            i_name = 'W_%i' % i   # worker name
            workers.append(Worker(i_name, i, GLOBAL_AC))
        saver = tf.train.Saver(max_to_keep=5)

    COORD = tf.train.Coordinator()
    if load_model==True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(SESS,ckpt.model_checkpoint_path)
    else:
        SESS.run(tf.global_variables_initializer())

    if OUTPUT_GRAPH:
        if os.path.exists(LOG_DIR):
            shutil.rmtree(LOG_DIR)
        tf.summary.FileWriter(LOG_DIR, SESS.graph)

    worker_threads = []
    for worker in workers:
        job = lambda: worker.work()
        t = threading.Thread(target=job)
        t.start()
        worker_threads.append(t)
    COORD.join(worker_threads)

    plt.plot(np.arange(len(GLOBAL_REWARD)), GLOBAL_REWARD)
    plt.xlabel('step')
    plt.ylabel('Total moving reward')
    plt.show()